In [ ]:
import pandas as pd
import psycopg2
import numpy as np
import os

In [ ]:
def create_connection(db_name, db_user, db_password, db_host, db_port):
    connection = None
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
        )
        print("Connection to PostgreSQL DB successful")
    except psycopg2.OperationalError as e:
        print(f"The error '{e}' occurred")
    return connection

In [ ]:
def create_database(connection, query):
    connection.autocommit = True
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Query executed successfully")
    except psycopg2.OperationalError as e:
        print(f"The error '{e}' occurred")

In [ ]:
def execute_query(connection, query):
    connection.autocommit = True
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("\rQuery executed successfully", end='')
    except psycopg2.OperationalError as e:
        print(f"The error '{e}' occurred")

In [ ]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        dd = pd.DataFrame(np.array(result))
        dd.columns = [desc[0] for desc in cursor.description]
        return dd #pd.DataFrame(np.array(result))
    except psycopg2.OperationalError as e:
        print(f"The error '{e}' occurred")

## Подключение к диску и БД

In [ ]:
if not os.path.exists('/content/drive/My Drive/Colab Notebooks/'):
  from google.colab import drive
  drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/love_sindrome/

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/love_sindrome


In [ ]:
db_set = dict()
file = open("db_connect.txt")
for line in file:
    k, v = line.strip().split('=')
    db_set[k] = v
file.close()
connection = create_connection(db_set['dbname'], db_set['uname'], db_set['pass'], db_set['server'], db_set['port'])
db_set = dict()

Connection to PostgreSQL DB successful


## Создание таблиц в БД

In [ ]:
#execute_query(connection, """DROP TABLE sindrom_trans""")
#execute_query(connection, """DROP TABLE sindrom_users""")
#execute_query(connection, """DROP TABLE sindrom_comp""")
#execute_query(connection, """DROP TABLE sindrom_rfm_table""")
#execute_query(connection, """DROP TABLE sindrom_chogort_table""")

In [ ]:
execute_query(connection, """
CREATE TABLE
IF NOT EXISTS sindrom_trans (
   id SERIAL PRIMARY KEY,
   date DATE,
   user_id INTEGER,
   comp_id INTEGER,
   donate REAL,
   comment TEXT,
   site TEXT,
   action TEXT,
   code TEXT,
   tr_m INTEGER,
   tr_y INTEGER,
   tr_ym INTEGER,
   tr_month TEXT
   )
""")

Query executed successfully

In [ ]:
execute_query(connection,  """
CREATE TABLE
IF NOT EXISTS sindrom_users (
   id SERIAL PRIMARY KEY,
   cr_date DATE,
   comp_id INTEGER,
   region TEXT,
   city TEXT,
   country TEXT,
   gender CHAR(10),
   subscr_id CHAR(40),
   en CHAR(10),
   vid CHAR(10),
   source TEXT,
   donor_type TEXT
   )
""")

Query executed successfully

In [ ]:
execute_query(connection, """
CREATE TABLE
IF NOT EXISTS sindrom_comp (
   id SERIAL PRIMARY KEY,
   cr_date DATE,
   partner_date DATE,
   partner TEXT,
   otrasl TEXT,
   type TEXT,
   sfera TEXT
   )
""")

Query executed successfully

## Чтение и предобработка данных из локальных файлов

In [ ]:
path = 'data/'
all = 0
rawdt = pd.DataFrame([])
for ff in os.listdir(path):
  filename, fileexp = os.path.splitext(ff)
  if fileexp == '.csv':
    min_date = pd.to_datetime(filename.split('-')[0], format='%d.%m.%Y')
    max_date = pd.to_datetime(filename.split('-')[1], format='%d.%m.%Y')
    #print(filename, min_date, max_date)
    dd = pd.read_csv(path + ff)
    dd['file'] = ff
    l = len(dd)
    all = all + l
    #Заполняем пропуски в `Контакт: ID` значенеим `Компания: ID` + 1000000
    dd['Компания: ID'] = dd['Компания: ID'].fillna(0)
    dd['Контакт: ID'] = dd['Контакт: ID'].fillna(dd['Компания: ID'] + 1000000)
    dd['Контакт: Дата создания'] = dd['Контакт: Дата создания'].fillna(dd['Компания: Дата создания'])
    dd['Контакт: Дата создания'] = dd['Контакт: Дата создания'].fillna(dd['Компания: Дата создания'])
    dd['Дата начала (Минимальное)'] = pd.to_datetime(dd['Дата начала (Минимальное)'], format='%d.%m.%Y')
    dd['Сумма в валюте сделки'] = dd['Сумма в валюте сделки']\
        .str.replace(' ', '')\
        .str.replace('\xa0','').astype('float')
    dd = dd.query('`Дата начала (Минимальное)` >= @min_date\
            and `Дата начала (Минимальное)` <= @max_date\
            and `Сумма в валюте сделки` > 10\
            and `В стадии: Удачно` == "Да"\
            and `Контакт: ID` > 0')
    rawdt = pd.concat([rawdt, dd])
    print(f'Загрузка {l} строк данных из файла {ff}')

print(f'Всего загружено {all} строк')
rawdt = rawdt.sort_values(by='Дата начала (Минимальное)').reset_index(drop=True)

<ipython-input-12-b92b058c8c62>:10: DtypeWarning: Columns (62,63,64,66,168,173,174,175,176,177,178,179,184,185,200,208,222,223,224) have mixed types. Specify dtype option on import or set low_memory=False.
  dd = pd.read_csv(path + ff)


Загрузка 8928 строк данных из файла 01.01.2024-30.06.2024.csv


<ipython-input-12-b92b058c8c62>:10: DtypeWarning: Columns (200,222) have mixed types. Specify dtype option on import or set low_memory=False.
  dd = pd.read_csv(path + ff)


Загрузка 7930 строк данных из файла 01.01.2023-30.06.2023.csv


<ipython-input-12-b92b058c8c62>:10: DtypeWarning: Columns (65,66,67,68,69,70,71,72,137,168,173,174,175,176,177,178,179,184,185,200,221,222) have mixed types. Specify dtype option on import or set low_memory=False.
  dd = pd.read_csv(path + ff)


Загрузка 8917 строк данных из файла 01.07.2023-31.12.2023.csv


<ipython-input-12-b92b058c8c62>:10: DtypeWarning: Columns (23,24,34,35,36,41,123,127,128,129,130,131,132,133,138,139,146,164,165,166) have mixed types. Specify dtype option on import or set low_memory=False.
  dd = pd.read_csv(path + ff)


Загрузка 8580 строк данных из файла 01.07.2024-31.12.2024.csv
Всего загружено 34355 строк


In [ ]:
tranz = rawdt[['ID', 'Дата начала (Минимальное)', 'Контакт: ID', 'Компания: ID',
               'Сумма в валюте сделки', 'Комментарий', 'Сайт - источник',
               'Action (строка)', 'Код (список)']].copy()
tranz = tranz.drop_duplicates()
tranz = tranz.reset_index(drop=True)
tranz.columns = ['id', 'date', 'user_id', 'comp_id', 'donate', 'comment', 'site',  'action', 'code']

In [ ]:
tranz['user_id'] = tranz['user_id'].astype('int')
tranz['comp_id'] = tranz['comp_id'].astype('int')
tranz['tr_m'] = tranz['date'].dt.month
tranz['tr_y'] = tranz['date'].dt.year
tranz['tr_ym'] = (tranz['tr_y'] - tranz['tr_y'].min()) * 12 + tranz['tr_m']
tranz['tr_month'] = tranz.apply(lambda row: f"{row['tr_y']}_{str(row['tr_m']):0>2}", axis=1)

tranz['code'] = tranz['code'].fillna('Не указано')
tranz['comment'] = tranz['comment'].fillna('Не указано')
tranz['action'] = tranz['action'].fillna('Не указано')
tranz['site'] = tranz['site'].fillna('Не указано')

tranz['site'] = tranz['site'].replace({'0' : 'Не указано'})
tranz['comment'] = tranz['comment'].str.strip()\
  .replace({'Благотворительный забег «СПОРТ ВО БЛАГО»' : 'Благотворительный забег «СПОРТ ВО БЛАГО»'})


In [ ]:
codelib = {'210' : 'Сборы на сторонних площадках (Вклад в будущее, Мэйлру, Нужна помощь и пр.)',
'211' : 'Частные доноры Рассылки',
'212' : 'Частные доноры Регулярные пожертвования',
'214' : 'Разовые пожертвования картой через сайт, ящики в фонде',
'215' : 'Пожертвования через СМС и спец ссылки',
'216' : 'Телемаркетинг',
'111' : 'ВИД Рассылки',
'112' : 'ВИД Регулярные списания',
'109' : 'ВИД (юрид. лица)',
'114' : 'ВИД массовые акции',
'208' : 'ВИД Мероприятия',
'209' : 'ВИД пожертвования',
'101' : 'Партнер велосотни',
'102' : 'Партнеры футбол',
'104' : 'Партнеры пробега и трейла',
'105' : 'Корпоративные пожертвования на программы',
'405' : 'Социальный маркетинг, пожертвования сотрудников (кроме АВБ)',
'201' : 'СВБ инд. учстники велосотня',
'204' : 'СВБ инд. учстники пробег и рейл',
'207' : 'АВБ',
'306' : 'Гранты, субсидии',
'517' : 'Магазин',
'519' : 'Консультационная деятельность',
'103' : 'Партнер(ы) лыжи',
'106' : 'Корпоративные гранты на программы',
'107' : 'Проекты корпоративных партнеров (спорт и не спорт) с потенциалом увеличения базы частных доноров',
'203' : 'СВБ инд. учстники лыжи',
'305' : 'Событийный фандрайзинг',
'401' : 'Внутренний корпоративный фандрайзинг по велосотне',
'402' : 'Внутренний корпоративный фандрайзинг по футболу',
'403' : 'Внутренний корпоративный фандрайзинг по лыжам',
'404' : 'Внутренний корпоративный фандрайзинг по пробегу и трейлу'}

tranz['code'] = tranz['code'].replace(codelib, regex=True).str.replace('f', '')

In [ ]:
tranz.head(5)

,id,date,user_id,comp_id,donate,comment,site,action,code,tr_m,tr_y,tr_ym,tr_month
0,332345,2023-01-01,406173,0,1000.0,"Разовое пожертвование в БФ ""Синдром любви""",sdl,Не указано,"Разовые пожертвования картой через сайт, ящики...",1,2023,1,2023_01
1,332331,2023-01-01,350353,0,300.0,"Ежемесячное пожертвование в БФ ""Синдром любви""",sdl,Не указано,Частные доноры Регулярные пожертвования,1,2023,1,2023_01
2,332333,2023-01-01,419745,0,500.0,"Ежемесячное пожертвование в БФ ""Синдром любви""",sdl,Не указано,Частные доноры Регулярные пожертвования,1,2023,1,2023_01
3,332337,2023-01-01,343691,0,300.0,"Ежемесячное пожертвование в БФ ""Синдром любви""",sdl,Не указано,Частные доноры Регулярные пожертвования,1,2023,1,2023_01
4,332339,2023-01-01,343285,0,1000.0,"Ежемесячное пожертвование в БФ ""Синдром любви""",sdl,Не указано,Частные доноры Регулярные пожертвования,1,2023,1,2023_01


In [ ]:
users = rawdt[['Контакт: ID', 'Контакт: Дата создания', 'Компания: ID',
               'Контакт: Регион_', 'Контакт: Город_', 'Контакт: Страна_',
               'Контакт: Пол', 'Контакт: ID подписки', 'Контакт: ЭН',
               'Контакт: Источник контакта', 'Контакт: Новый донор тип', 'Контакт: ВИД']].copy()
users = users.dropna(subset=['Контакт: Дата создания'])
users = users.drop_duplicates()
users = users.reset_index(drop=True)
users.columns = ['id', 'cr_date', 'comp_id', 'region', 'city', 'country', 'gender', 'subscr_id', 'en', 'source', 'donor_type', 'vid']

In [ ]:
users['cr_date'] = pd.to_datetime(users['cr_date'], format='%d.%m.%Y')
users['comp_id'] = users['comp_id'].fillna(0)
users['gender'] = users['gender'].fillna('Не указано')
users['source'] = users['source'].fillna('Не указано')
users['gender'] = users['gender'].fillna('Не указано')
users['city'] = users['city'].fillna('Не указано')
users['region'] = users['region'].fillna('Не указано')
users['country'] = users['country'].fillna('Не указано')
users['donor_type'] = users['donor_type'].fillna('Не указано')
users['subscr_id'] = users['subscr_id'].fillna('Не указано')

users['gender'] = users['gender'].replace({'0' : 'Не указано'})
users['city'] = users['city'].replace(r'.*Моск.*', 'Москва', regex=True)
users['city'] = users['city'].replace(r'.*моск.*', 'Москва', regex=True)
users['city'] = users['city'].str.replace(r'^г.', '', regex=True)\
    .str.replace(r'^д.', '', regex=True).str.replace(r'^пос.', '', regex=True)\
    .str.replace(r'^п.', '', regex=True).str.replace(r'^с.', '', regex=True)\
    .str.replace(r'^c.', '', regex=True).str.replace(r'^ст.', '', regex=True)\
    .str.strip()
users['region'] = users['region'].replace(r'.*Моск.*', 'Москва', regex=True)

In [ ]:
users.head(5)

,id,cr_date,comp_id,region,city,country,gender,subscr_id,en,source,donor_type,vid
0,406173.0,2022-08-03,0.0,Республика Татарстан,Казань,Россия,Не указано,Не указано,Да,Не указано,Не указано,Нет
1,350353.0,2022-08-03,0.0,Не указано,Не указано,Россия,Не указано,sc_cfa2311470bc523e0ea784bd9460d,Да,Не указано,WL СЛ,Нет
2,419745.0,2022-08-03,0.0,Москва,Москва,Россия,Не указано,sc_06e565456d9e2edfa0a1fb77b10ad,Да,Не указано,Не указано,Нет
3,343691.0,2022-08-03,0.0,Не указано,Не указано,Россия,Не указано,sc_278c179158ea47580547c89d5b914,Да,Other,WL СЛ,Нет
4,343285.0,2022-08-03,0.0,Не указано,Не указано,Россия,Не указано,sc_17cae519bcda8ae9a9cefa8c36788,Да,Meropr,Не указано,Да


In [ ]:
comp = rawdt.query('`Компания: ID` != 0')[['Компания: ID', 'Компания: Дата создания',
               'Компания: Дата начала сотрудничества', 'Компания: Статус партнёра',
               'Компания: Отрасль', 'Компания: Тип', 'Компания: Сфера деятельности']].copy()
comp = comp.drop_duplicates()
comp = comp.reset_index(drop=True)
comp.columns = ['id', 'cr_date', 'partner_date', 'partner', 'otrasl', 'type', 'sfera']

In [ ]:
comp['cr_date'] = pd.to_datetime(comp['cr_date'], format='%d.%m.%Y')
comp['partner_date'] = pd.to_datetime(comp['partner_date'], format='%d.%m.%Y')

comp['partner'] = comp['partner'].fillna('Не указано')
comp['otrasl'] = comp['otrasl'].fillna('Не указано')
comp['type'] = comp['type'].fillna('Не указано')
comp['partner_date'] = comp['partner_date'].fillna('01.01.1900')

In [ ]:
comp.head()

,id,cr_date,partner_date,partner,otrasl,type,sfera
0,57013.0,2022-07-22,1900-01-01,Активный,Не указано,Корпоративный партнёр,NaN
1,62201.0,2023-07-05,1900-01-01,Активный,Не указано,Other,NaN
2,62101.0,2022-08-16,2018-12-28,Активный,Не указано,НКО,NaN
3,62185.0,2023-03-23,2023-01-01,Активный,Не указано,Корпоративный партнёр,Другое
4,62183.0,2023-03-23,2023-01-01,Активный,Не указано,Корпоративный партнёр,Другое


Загрузка данных в БД PostgreSQL

In [ ]:
#execute_query(connection, "TRUNCATE TABLE sindrom_trans;")
#execute_query(connection, "TRUNCATE TABLE sindrom_users;")
#execute_query(connection, "TRUNCATE TABLE sindrom_comp;")

In [ ]:
len_trans_table = execute_read_query(connection, """SELECT COUNT(*) FROM sindrom_trans;""").loc[0,'count']
len_users_table = execute_read_query(connection, """SELECT COUNT(*) FROM sindrom_users;""").loc[0,'count']
len_comp_table = execute_read_query(connection, """SELECT COUNT(*) FROM sindrom_comp;""").loc[0,'count']
print(f'Количество записей в таблице транзакций {len_trans_table}')
print(f'Количество записей в таблице пользователей {len_users_table}')
print(f'Количество записей в таблице компаний {len_comp_table}')

Количество записей в таблице транзакций 22072
Количество записей в таблице пользователей 8598
Количество записей в таблице компаний 36


In [ ]:
#количество строк экспортируемых за раз
inrows = 1000

In [ ]:
for l in range(len(tranz) // inrows + 1):
#for l in range(3):
  insert_row = """INSERT INTO
    sindrom_trans (id, date, user_id, comp_id, donate, comment, site,  action, code,
                tr_m, tr_y, tr_ym, tr_month)
  VALUES"""
  cols = tranz.columns
  tt = tranz.iloc[l*inrows:l*inrows + inrows]
  for i in range(len(tt)):
    istr = '('
    for c in cols:
      if tt[c].dtypes != 'int' and tt[c].dtypes != 'float64':
        istr += "'" + str(tt.iloc[i][c]).replace("'", "") + "', "
      else:
        istr += str(tt.iloc[i][c]) + ', '
    istr = istr[:-2]
    istr += '), '+"\r\n"
    insert_row += istr
  insert_row = insert_row[:-4]
  insert_row += ' ON CONFLICT DO NOTHING ;'
  execute_query(connection, insert_row)
  print(f'\r Экспорт в БД в таблицу sindrom_trans строк {l*inrows} - {l*inrows + inrows} прошeл успешно', end='', flush=True)
print('\r\nВСЕ!')

 Экспорт в БД в таблицу sindrom_trans строк 22000 - 23000 прошeл успешно
ВСЕ!


In [ ]:
for l in range(len(users) // inrows + 1):
#for l in range(3):
  insert_row = """INSERT INTO
    sindrom_users (id, cr_date, comp_id, region, city, country, gender, subscr_id,
                en, source, donor_type, vid)
  VALUES"""
  cols = users.columns
  tt = users.iloc[l*inrows:l*inrows + inrows]
  for i in range(len(tt)):
    istr = '('
    for c in cols:
      if tt[c].dtypes != 'int' and tt[c].dtypes != 'float64':
        istr += "'" + str(tt.iloc[i][c]).replace("'", "") + "', "
      else:
        istr += str(tt.iloc[i][c]) + ', '
    istr = istr[:-2]
    istr += '), '+"\r\n"
    insert_row += istr
  insert_row = insert_row[:-4]
  insert_row += ' ON CONFLICT DO NOTHING ;'
  execute_query(connection, insert_row)
  print(f'\r Экспорт в БД в таблицу sindrom_users строк {l*inrows} - {l*inrows + inrows} прошeл успешно', end='', flush=True)
print('\r\nВСЕ!')

 Экспорт в БД в таблицу sindrom_users строк 8000 - 9000 прошeл успешно
ВСЕ!


In [ ]:
for l in range(len(users) // inrows + 1):
  insert_row = """INSERT INTO
    sindrom_comp (id, cr_date, partner_date, partner, otrasl, type, sfera)
  VALUES"""
  cols = comp.columns
  tt = comp.iloc[l*inrows:l*inrows + inrows]
  for i in range(len(tt)):
    istr = '('
    for c in cols:
      if tt[c].dtypes != 'int' and tt[c].dtypes != 'float64':
            istr += "'" + str(tt.iloc[i][c]).replace("'", "") + "', "
      else:
        istr += str(tt.iloc[i][c]) + ', '
    istr = istr[:-2]
    istr += '), '+"\r\n"
    insert_row += istr
  insert_row = insert_row[:-4]
  insert_row += ' ON CONFLICT DO NOTHING ;'
  if len(tt) > 0:  execute_query(connection, insert_row)
  print(f'\r Экспорт в БД в таблицу sindrom_comp строк {l*inrows} - {l*inrows + inrows} прошeл успешно', end='', flush=True)
print('\r\nВСЕ!')

 Экспорт в БД в таблицу sindrom_comp строк 8000 - 9000 прошeл успешно
ВСЕ!


In [ ]:
len_trans_table = execute_read_query(connection, """SELECT COUNT(*) FROM sindrom_trans;""").loc[0,'count']
len_users_table = execute_read_query(connection, """SELECT COUNT(*) FROM sindrom_users;""").loc[0,'count']
len_comp_table = execute_read_query(connection, """SELECT COUNT(*) FROM sindrom_comp;""").loc[0,'count']
print(f'Количество записей в таблице транзакций {len_trans_table}')
print(f'Количество записей в таблице пользователей {len_users_table}')
print(f'Количество записей в таблице компаний {len_comp_table}')

Количество записей в таблице транзакций 22072
Количество записей в таблице пользователей 8598
Количество записей в таблице компаний 36


## Агрегация загруженных данных в PostgreSQL

In [ ]:
execute_query(connection, """ALTER TABLE sindrom_users
  ADD COLUMN IF NOT EXISTS donate_sum REAL,
  ADD COLUMN IF NOT EXISTS tr_count INTEGER,
  ADD COLUMN IF NOT EXISTS first_date DATE,
  ADD COLUMN IF NOT EXISTS last_date DATE""")

Query executed successfully

In [ ]:
execute_query(connection, """UPDATE sindrom_users
SET donate_sum = t2.donate_sum, tr_count = t2.tr_count, first_date = t2.first_date, last_date = t2.last_date
FROM (
  SELECT user_id, SUM(donate) as donate_sum, COUNT(id) as tr_count, MIN(date) as first_date, MAX(date) as last_date
  FROM sindrom_trans
  GROUP BY user_id
  ) as t2
WHERE sindrom_users.id = t2.user_id
  """)

Query executed successfully

In [ ]:
execute_query(connection, """ALTER TABLE sindrom_users
  ADD COLUMN IF NOT EXISTS days_on INTEGER,
  ADD COLUMN IF NOT EXISTS days_ago INTEGER,
  ADD COLUMN IF NOT EXISTS oper_freq REAL,
  ADD COLUMN IF NOT EXISTS user_type CHAR(10)""")

Query executed successfully

In [ ]:
execute_query(connection, """UPDATE sindrom_users SET days_on = last_date - first_date""")
execute_query(connection, """UPDATE sindrom_users SET days_ago = MAKE_DATE(2024, 12, 31) - last_date""")
execute_query(connection, """UPDATE sindrom_users SET oper_freq = tr_count / (days_on::NUMERIC / 30 + 1)""")

Query executed successfully

In [ ]:
execute_query(connection, """UPDATE sindrom_users
    SET user_type = CASE
            WHEN comp_id <> 0 THEN 'Компания'
            WHEN days_on = 0 AND comp_id = 0 THEN 'Разовый'
            WHEN subscr_id <> 'Не указано' AND comp_id = 0 THEN 'Подписчик'
          ELSE 'Простой'
        END""")

Query executed successfully

In [ ]:
execute_read_query(connection, """SELECT * FROM sindrom_users WHERE donate_sum > 5000 LIMIT 10""")

,id,cr_date,comp_id,region,city,country,gender,subscr_id,en,vid,...,last_date,days_on,days_ago,oper_freq,user_type,rfm,rfm_r,rfm_f,rfm_m,chogort
0,437169,2022-08-03,0,Не указано,Москва,RUS,Не указано,sc_33d989067f79d210034f9f4b89389,Да,Да,...,2023-12-24,335,373,0.90411,Подписчик,r3f2m1,r3,f2,m1,2023-01-01
1,419517,2022-08-03,0,Москва,Москва,Россия,Не указано,sc_747e5806879720886dd3276427cad,Да,Да,...,2024-10-23,640,69,0.895522,Подписчик,r1f2m1,r1,f2,m1,2023-01-01
2,342333,2022-08-03,0,Москва,Москва,Россия,Не указано,Не указано,Да,Да,...,2024-10-17,622,75,0.092025,Простой,r1f3m1,r1,f3,m1,2023-02-01
3,437225,2022-08-03,0,Не указано,Москва,RUS,Не указано,sc_90c58811ed159227adcf8b29a1048,Да,Нет,...,2024-11-17,670,44,0.685714,Подписчик,r1f3m1,r1,f3,m1,2023-01-01
4,437309,2022-08-03,0,Не указано,Не указано,RUS,Не указано,sc_cffc6469836285690901258246211,Да,Нет,...,2024-11-25,676,36,1.01983,Подписчик,r1f2m1,r1,f2,m1,2023-01-01
5,393153,2022-08-03,0,Москва,Москва,Россия,Не указано,sc_d9c854e6f66a322df9b18cc6ac8c9,Да,Нет,...,2024-12-01,700,30,0.945205,Подписчик,r1f2m1,r1,f2,m1,2023-01-01
6,477733,2023-10-07,0,Не указано,Москва,Не указано,М,sc_8eaffdb0d6b821fb3fc267e35f339,Нет,Да,...,2024-12-07,427,24,0.853392,Подписчик,r1f2m1,r1,f2,m1,2023-10-01
7,489893,2024-07-19,0,Не указано,Москва,Не указано,М,Не указано,Нет,Да,...,2024-07-19,0,165,1.0,Разовый,r2f2m1,r2,f2,m1,2024-07-01
8,486739,2024-04-08,0,Не указано,Ступино,Не указано,М,sc_33aa339763fa70323d0c45854b623,Нет,Да,...,2024-12-31,276,0,0.980392,Подписчик,r1f2m1,r1,f2,m1,2024-03-01
9,399817,2022-08-03,0,Москва,Москва,Россия,Не указано,sc_1a221f213e422e54d164a2e7f608f,Да,Нет,...,2024-07-23,555,161,0.717949,Подписчик,r2f2m1,r2,f2,m1,2023-01-01


## RFM витрина

In [ ]:
execute_query(connection, """ALTER TABLE sindrom_users
  ADD COLUMN IF NOT EXISTS rfm CHAR(15),
  ADD COLUMN IF NOT EXISTS rfm_r CHAR(5),
  ADD COLUMN IF NOT EXISTS rfm_f CHAR(5),
  ADD COLUMN IF NOT EXISTS rfm_m CHAR(5)
  """)

Query executed successfully

In [ ]:
execute_query(connection, """UPDATE sindrom_users
    SET rfm_r = CASE
            WHEN days_ago < 90 THEN 'r1'
            WHEN days_ago > 180 THEN 'r3'
            ELSE 'r2'
            END,
    rfm_f = CASE
            WHEN oper_freq > 2 THEN 'f1'
            WHEN oper_freq < 0.7 THEN 'f3'
            ELSE 'f2'
            END,
    rfm_m = CASE
            WHEN donate_sum > 5000 THEN 'm1'
            WHEN donate_sum < 1400 THEN 'm3'
            ELSE 'm2'
            END
        """)

Query executed successfully

In [ ]:
execute_query(connection, """UPDATE sindrom_users SET rfm = rfm_r || rfm_f || rfm_m""")

Query executed successfully

In [ ]:
execute_query(connection, """DROP TABLE IF EXISTS sindrom_rfm_table""")
execute_query(connection, """
CREATE TABLE
IF NOT EXISTS sindrom_rfm_table (
   rfm_r CHAR(5),
   rfm_f CHAR(5),
   rfm_m CHAR(5),
   donate_sum REAL,
   tr_count INTEGER,
   user_count INTEGER
   )
""")

Query executed successfully

In [ ]:
execute_query(connection, """TRUNCATE TABLE sindrom_rfm_table""")
execute_query(connection, """INSERT INTO sindrom_rfm_table (
    SELECT
      rfm_r, rfm_f, rfm_m,
      SUM(donate_sum) as donate_sum,
      SUM(tr_count) as tr_counr,
      COUNT(DISTINCT id) as user_count
    FROM sindrom_users
    GROUP BY rfm_r, rfm_f, rfm_m)""")

Query executed successfully

In [ ]:
execute_read_query(connection, """SELECT * FROM sindrom_rfm_table""")

,rfm_r,rfm_f,rfm_m,donate_sum,tr_count,user_count
0,r1,f1,m1,535987.0,588,13
1,r1,f1,m2,11920.0,19,3
2,r1,f1,m3,4273.0,20,5
3,r1,f2,m1,12499812.0,6804,372
4,r1,f2,m2,758225.0,1408,229
5,r1,f2,m3,255005.0,591,430
6,r1,f3,m1,10551377.0,862,140
7,r1,f3,m2,356207.0,572,122
8,r1,f3,m3,38091.0,133,45
9,r2,f1,m1,96100.0,20,6


## Витрина когорт

In [ ]:
#execute_query(connection, """ALTER TABLE sindrom_users DROP COLUMN chogort""")
#execute_query(connection, """ALTER TABLE sindrom_trans DROP COLUMN chogort""")

In [ ]:
execute_query(connection, """ALTER TABLE sindrom_users ADD COLUMN IF NOT EXISTS chogort DATE""")

Query executed successfully

In [ ]:
#execute_query(connection, """UPDATE sindrom_users
#  SET chogort = date_part('year', first_date)::varchar(5)
#  || '_' ||
#  TO_CHAR(date_part('month', first_date), 'fm00')""")

execute_query(connection, """UPDATE sindrom_users SET chogort = DATE_TRUNC('month', first_date)""")

Query executed successfully

In [ ]:
execute_query(connection, """ALTER TABLE sindrom_trans ADD COLUMN IF NOT EXISTS chogort DATE""")
execute_query(connection, """ALTER TABLE sindrom_trans ADD COLUMN IF NOT EXISTS m_live INTEGER""")

Query executed successfully

In [ ]:
execute_query(connection, """UPDATE sindrom_trans as t
SET chogort = u.chogort
FROM sindrom_users as u
WHERE t.user_id = u.id """)

Query executed successfully

In [ ]:
#execute_query(connection, """UPDATE sindrom_trans SET m_live = DIV(date - chogort, 30) + 1 """)

execute_query(connection, """UPDATE sindrom_trans
  SET m_live = EXTRACT(YEAR FROM AGE(date, chogort)) * 12 +
            EXTRACT(MONTH FROM AGE(date, chogort)) + 1 """)


Query executed successfully

In [ ]:
execute_query(connection, """DROP TABLE IF EXISTS sindrom_chogort_table""")

execute_query(connection, """
CREATE TABLE
IF NOT EXISTS sindrom_chogort_table (
   chogort DATE,
   m_live INTEGER,
   donate_sum REAL,
   tr_count INTEGER,
   user_count INTEGER
   )
   """)

Query executed successfully

In [ ]:
execute_query(connection, """TRUNCATE TABLE sindrom_chogort_table""")

execute_query(connection, """INSERT INTO sindrom_chogort_table (
    SELECT chogort,
      m_live,
      SUM(donate),
      COUNT(id),
      COUNT(DISTINCT user_id)
    FROM sindrom_trans
    WHERE m_live > 0
    GROUP BY chogort, m_live
  )"""
)

Query executed successfully

In [ ]:
execute_read_query(connection, """SELECT
chogort, m_live, donate_sum,
SUM(donate_sum)
      OVER (PARTITION BY chogort
            ORDER BY m_live) AS cumsum
FROM sindrom_chogort_table""")

,chogort,m_live,donate_sum,cumsum
0,2023-01-01,1,3267368.0,3267368.0
1,2023-01-01,2,280780.0,3548148.0
2,2023-01-01,3,348372.0,3896520.0
3,2023-01-01,4,272135.0,4168655.0
4,2023-01-01,5,347729.0,4516384.0
...,...,...,...,...
295,2024-10-01,2,119450.0,871215.0
296,2024-10-01,3,70606.0,941821.0
297,2024-11-01,1,168793.0,168793.0
298,2024-11-01,2,2513.0,171306.0


In [ ]:
execute_query(connection, """ALTER TABLE sindrom_chogort_table ADD COLUMN IF NOT EXISTS cumsum REAL""")
execute_query(connection, """ALTER TABLE sindrom_chogort_table ADD COLUMN IF NOT EXISTS rr REAL""")

execute_query(connection, """UPDATE sindrom_chogort_table as c
  SET cumsum = w.cumsum,
  rr = w.rr
  FROM
      (
      SELECT
      chogort, m_live, donate_sum,
      SUM(donate_sum)
            OVER (PARTITION BY chogort
                  ORDER BY m_live) AS cumsum,
      user_count::NUMERIC / MAX(user_count)
            OVER (PARTITION BY chogort
                  ORDER BY m_live) AS rr
      FROM sindrom_chogort_table
      ) as w
  WHERE c.chogort = w.chogort AND c.m_live = w.m_live
""")

Query executed successfully

In [ ]:
execute_read_query(connection, """SELECT * FROM sindrom_chogort_table""")

,chogort,m_live,donate_sum,tr_count,user_count,cumsum,rr
0,2023-01-01,1,3267368.0,387,363,3267368.0,1.0
1,2023-01-01,2,280780.0,297,269,3548148.0,0.741047
2,2023-01-01,3,348372.0,346,278,3896520.0,0.76584
3,2023-01-01,4,272135.0,291,261,4168655.0,0.719008
4,2023-01-01,5,347729.0,286,258,4516384.0,0.710744
...,...,...,...,...,...,...,...
295,2024-10-01,2,119450.0,19,17,871215.0,0.100592
296,2024-10-01,3,70606.0,11,10,941821.0,0.059172
297,2024-11-01,1,168793.0,109,95,168793.0,1.0
298,2024-11-01,2,2513.0,11,9,171306.0,0.094737
